**Chapter 8 – MLP**

In [1]:
#importatation de modules commun

import numpy as np
import pandas as pd

# pour la énération de seed
np.random.seed(4)

# pur l'afficae de figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from time import *


In [2]:
#lecture des deux datasets Train_FraudeAA et Test_FraudeAA
#le dataset Original de 500 insatnces a été split en jeu d'entrainement 80%

data_train = pd.read_csv("Train_FraudeAA.CSV")
data_test = pd.read_csv("Test_FraudeAA.CSV")

Transformation des variables qualitatives (object) en type vecteur nombre. 
Exigence de scikit-learn

Au lieu de passer par l'encoder OnehotEncoder, voici une approche plus simple dans ce contexte de l'exemple
C'est d'utiliser la fonction get_dummy() de pandas qui produit le même résultat que l'encoder OneHotEncoder..

Nous avons 2 variables qualitatives:
1. InjuryType = BrokenLimb,Soft Tissue,Back,Serious
2. OvernightHospitalStay=yes/no




In [3]:
nouv_df_train=pd.get_dummies(data_train[data_train.columns[:-1]])

In [4]:
#Nous allons compléter le dataFrame nouv_df avec la variable cible dont le nom
#doit être class
nouv_df_train['class'] = data_train['Fraude']

In [5]:
# Séparer des données des variables prédictives (X) 
# de la variable cible (Y)
X_train=nouv_df_train.iloc[:,:-1]
y_train=nouv_df_train['class']

In [6]:
#normalisation au cas ou
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)   

# Entrainement

In [7]:

#importation du module mlp

from sklearn.neural_network import MLPClassifier

#utilsation 2 couches cachées. La premiere est composé de 100 neurones et l'autre de 50.
#la fonction d'activation est le relu
#la regle d'apprentissage est la descente du gradient: solver=sgd

clf = MLPClassifier(hidden_layer_sizes=(100, 50), solver='sgd', max_iter=12500, activation='relu', random_state=1)

#mesure du temps courant
tc = clock()
clf.fit(X_train,y_train)
tt = clock()
print("le temps d'entrainement en secondes", tt-tc)


le temps d'entrainement en secondes 10.74462418483876


## Validation avec un jeu de test

Validation du classeur clf avec le jeu de test.
Il faut s'assurer que le jeu de test est dans la même configuration
que le jeu d'entrainement.
Étant que nous avons transformé les variables qualitativesen en nombre,
Dans le jeu d'entrainement:
1.InjuryType = BrokenLimb,Soft Tissue,Back,Serious
2.OvernightHospitalStay=yes/no
Il faut faire la même chose dans le cas de jeu de test:


In [8]:
nouv_df_test=pd.get_dummies(data_test[data_test.columns[:-1]])

In [9]:
#il faut aussi renommer la variable cible
# et comme la variable cible est nommée Fraude, il faut aussi la renommer en class
nouv_df_test['class'] = data_test['Fraude']

In [10]:
#séparons les donnees de test des variables prédictives de celles 
#de la variable cible et normaliser aussi.

X_test=nouv_df_test.iloc[:,:-1]
y_test=nouv_df_test['class']
X_test = scaler.transform(X_test) 

In [11]:
#Voici un exemple de résultat de test avec le modele clf obtenu initialement 
# testé avec les données de test

from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

# le taux de détection est de l'ordre de :


0.9347826086956522

In [12]:
#Calibarage du MLP pour la recherche des valeurs optimales des hyperparamètres:
#nombre de couches cachées et le nombre de neurones associées : tuples : ()
#la meilleure fonction d'activation entre le relu, la tengente hyperbolique
#la meilleure regle d'apprentissage
#le nombre d'iteration
#le pas d'apprentissage

mlp = MLPClassifier()

parameter_space = {
    'hidden_layer_sizes': [(50,50), (100, 100), (50,100,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'max_iter':[100,2500],
    'learning_rate': ['constant','adaptive'],
}

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

clf = RandomizedSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
#clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=MLPClassifier(), n_jobs=-1,
                   param_distributions={'activation': ['tanh', 'relu'],
                                        'hidden_layer_sizes': [(50, 50),
                                                               (100, 100),
                                                               (50, 100, 50)],
                                        'learning_rate': ['constant',
                                                          'adaptive'],
                                        'max_iter': [100, 2500],
                                        'solver': ['sgd', 'adam']})

In [15]:
# meilleur parametre
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'solver': 'adam', 'max_iter': 2500, 'learning_rate': 'constant', 'hidden_layer_sizes': (100, 100), 'activation': 'tanh'}


In [16]:
#test
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_test, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.98      0.95      0.96        56
           1       0.92      0.97      0.95        36

    accuracy                           0.96        92
   macro avg       0.95      0.96      0.95        92
weighted avg       0.96      0.96      0.96        92



In [ ]:
#on passe de 93% à 95% : u une bonne amélioration...